In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import cftime
import yaml
import calendar
import xarray as xr
import rpy2.robjects as ro

#My functions
sys.path.insert(0,'../functions/')
import functions_HeatWavesCities as fun_HWC


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_EUR-11.txt', 'r') as file:  path_eur11 = file.read()
    
dir_CORDEX     = path_eur11
dir_CORDEX_out = f'{path_main}Data/EURO-CORDEX/'
dir_scripts    = f'{path_main}Scripts/'
dir_names      = f'{path_main}Scripts/Model_lists/'
dir_functions  = f'{path_main}Scripts/functions/'

## Prepare variables and parameters

In [ ]:
#Source R functions
r = ro.r
r.source(dir_scripts + 'functions/get_HWMId_vCities_NxN.r')

#Define scenarios and variables
scenarios = ['historical', 'rcp85']
RCPs      = ['rcp85']
variables = ['tasmax']
variables_out = ['TX']

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp26'] = []
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP26.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp26'].append(eval(line[:-1]))
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))

#Add models for historical
mod_85 = ["_".join(model) for model in all_models['rcp85']]
mod_26 = ["_".join(model) for model in all_models['rcp26']]
all_models['historical'] = [model.split('_') for model in sorted(list(set(mod_85).union(set(mod_26))))]


## HWMId for all of Europe

In [ ]:
#Define output folder
dir_HWMId_out = dir_CORDEX_out + 'HWMId_Europe/'
if not os.path.exists(dir_HWMId_out): os.mkdir(dir_HWMId_out)
    
#Define reference and application years
app_years = [1981, 2099]
ref_years_vec = ro.r.c(1981, 2010)
app_years_vec = ro.r.seq(app_years[0], app_years[1])
years_cdo = str(app_years[0]) + '/' + str(app_years[-1])

#Loop over RCPs
for RCP in RCPs:
    
    #Define scenarios and select models
    scenarios = ['historical', RCP]
    models    = all_models[RCP]
    
    #Loop over models
    for model in models:
        
        print(" -" + "_".join(model))

        #Loop over HSIs
        for (var_in, var_out) in zip(variables, variables_out):

            #Loop over scenarios
            files_merge =[]
            for i1, scen in enumerate(scenarios):

                #Get file name
                dir_data = dir_CORDEX + '/' + scen + '/' + var_in + '/'
                files_read = [dir_data + file for file in os.listdir(dir_data) if scen in file and model[0] in file and model[1] in file and model[2] in file]
                if len(files_read)==0:  sys.exit('No files found')

                #Select only years after 1981
                if scen=='historical':  files_read = [file for file in files_read if int(file[-11:-7])>=1981]

                #Collect all files (convert temperature from °C to K for ALADIN53 in RCP85)
                if 'ALADIN53' in "_".join(model) and scen=='rcp85' and var_in=='tasmax':
                    fname_ALA_tmp1 = dir_CORDEX + 'tmp/ALADIN53_correct1.nc'
                    fname_ALA_tmp2 = dir_CORDEX + 'tmp/ALADIN53_correct2.nc'
                    os.system('cdo mergetime ' + " ".join(sorted(files_read)) + ' ' + fname_ALA_tmp1)
                    os.system('cdo addc,+273.15 ' + fname_ALA_tmp1 + ' ' + fname_ALA_tmp2)
                    files_merge = files_merge + [fname_ALA_tmp2]
                    os.remove(fname_ALA_tmp1)
                    
                else:
                    files_merge = files_merge + sorted(files_read)
                
            #Merge time
            dir_tmp = dir_CORDEX + 'tmp/'
            fname_tmp1 = dir_tmp + var_in + '_' + "_".join(model) + '_' + RCP + '_tmp1.nc'
            fname_tmp2 = dir_tmp + var_in + '_' + "_".join(model) + '_' + RCP + '_tmp2.nc'
            fname_tmp3 = fname_tmp2
            if os.path.exists(fname_tmp1):  os.remove(fname_tmp1)
            if os.path.exists(fname_tmp2):  os.remove(fname_tmp2)
            os.system('cdo mergetime ' + " ".join(files_merge) + ' ' + fname_tmp1)
            os.system('cdo selyear,' + years_cdo + ' ' + fname_tmp1 + ' ' + fname_tmp2)

            #Get lon and lat name
            data_check = xr.open_dataset(fname_tmp2)
            if 'rlat' in data_check.dims:
                coord_names = ['rlon', 'rlat']
                coord_names_R = ro.r.c('rlon', 'rlat')
            elif 'x' in data_check.dims:
                coord_names = ['x', 'y']
                coord_names_R = ro.r.c('x', 'y')
            
            #Define input filename
            fname_in = fname_tmp2
            
            #Check if last year is complete
            DOY_max = data_check.time.dt.dayofyear.max().item()
            DOY_end = data_check.time.dt.dayofyear[-1].item()
            Yr_end  = data_check.time.dt.year[-1].item()
            if DOY_max==366:  DOY_max = DOY_max - (not calendar.isleap(Yr_end)) #Adjust maximum DOY for non-leap years
                
            #Special treatment if last year is not complete
            if DOY_end<DOY_max:
                
                print('Last year ' + str(Yr_end) + ' is not complete. Dropping ' + str(Yr_end))

                #Sub-select data if last year is not complete and save in file
                years_cdo2 = years_cdo.split('/')[0] + '/' + str(Yr_end-1)
                fname_tmp3 = dir_tmp + var_in + '_' + "_".join(model) + '_' + RCP + '_tmp3.nc'
                os.system('cdo selyear,' + years_cdo2 + ' ' + fname_tmp2 + ' ' + fname_tmp3)
                
                #Change input filename
                fname_in = fname_tmp3
                
                #Re-read data for check
                data_check = xr.open_dataset(fname_tmp3)
                
            #Adjust application time (if model simualtion does not go until 2099)
            if data_check.time.dt.year[-1].item()<app_years[1]:
                app_years_HWMId = ro.r.seq(app_years[0], data_check.time.dt.year[-1].item())
            else:
                app_years_HWMId = app_years_vec
            
            #Create time vector for HWMId
            time_HWMID = xr.cftime_range(start=str(app_years_HWMId[0]) + '0101', end=str(app_years_HWMId[-1]) + '1231', freq='Y', calendar='standard')
            
            #Calculate HWMId
            t_start = t_util.time()
            N_cores = 5
            output = r.get_HWMId_vCities_NxN(var_in, fname_in, ref_years_vec, app_years_HWMId, coord_names_R, dir_functions, N_cores)

            #Extract variables from R output (dims: lon, lat, time)
            HWMID_strength = np.array(output.rx2("HWMID_strength"))
            HWMID_length   = np.array(output.rx2("HWMID_length"))
            HWMID_DOYstart = np.array(output.rx2("HWMID_DOYstart"))

            #Create dataset with same coords as original dataset
            data_HWMID = xr.Dataset(coords=dict(zip(data_check.coords, [data_check[coord] for coord in data_check.coords])))
            if 'height' in data_HWMID.coords: data_HWMID = data_HWMID.drop('height')

            #Put yearly time axis
            data_HWMID = data_HWMID.isel(time=1).drop('time')
            data_HWMID = data_HWMID.assign_coords({'time': time_HWMID})

            #Put HWMID
            data_HWMID['HWMID'] = (coord_names + ['time'], HWMID_strength)
            data_HWMID['HWMID_length'] = (coord_names + ['time'], HWMID_length)
            data_HWMID['HWMID_DOYstart'] = (coord_names + ['time'], HWMID_DOYstart)

            #Make sure that the dimension order is correct
            if 'x' in data_HWMID.dims:       lat_name, lon_name = 'y', 'x'
            elif 'rlat' in data_HWMID.dims:  lat_name, lon_name = 'rlat', 'rlon'
            else:                            sys.exit('Names for lat and lon undefined.')  
            data_HWMID = data_HWMID.transpose(lat_name, lon_name, 'time')

            #Save in file
            fname_HWMId = dir_HWMId_out + 'HWMId-' + var_out + "_" + "_".join(model) + '_' + "-".join(scenarios) + "_" + str(app_years[0]) + "-" + str(app_years[1]) + ".nc"
            if os.path.exists(fname_HWMId): os.remove(fname_HWMId)
            data_HWMID.to_netcdf(fname_HWMId)

            t_stop = t_util.time()
            print('Time elapsed:' + '{:.0f}'.format(t_stop - t_start))
            
            #Remove temporary files
            if os.path.exists(fname_tmp1):      os.remove(fname_tmp1)
            if os.path.exists(fname_tmp2):      os.remove(fname_tmp2)
            if os.path.exists(fname_tmp3):      os.remove(fname_tmp3)
            if os.path.exists(fname_ALA_tmp2):  os.remove(fname_ALA_tmp2)
